In [2]:
import os
import pyarrow.parquet as pq
import geopandas as gpd
import pandas as pd
import datetime
import json

def er_geoparquet(filsti):
    """Sjekker om en parquet-fil er en GeoParquet-fil"""
    try:
        # Les metadata fra parquet-filen
        metadata = pq.read_metadata(filsti)
        
        # Sjekk om metadata inneholder noen geometri-kolonner
        schema = pq.read_schema(filsti)
        for navn in schema.names:
            felt = schema.field(navn)
            # Sjekk metadata for kolonnen
            if felt.metadata and ('geo' in felt.metadata or b'geo' in felt.metadata):
                return True
                
        # Les filen med geopandas og se om den inneholder en geometrikolonne
        try:
            gdf = gpd.read_parquet(filsti)
            if hasattr(gdf, 'geometry') and gdf.geometry.name in gdf.columns:
                return True
        except Exception:
            pass
            
        return False
    except Exception:
        return False

def konverter_til_geoparquet(kildefilsti, målmappesti=None):
    """Konverterer ulike geografiske formater til GeoParquet-format"""
    try:
        # Få filnavn uten endelse
        base_navn = os.path.basename(kildefilsti)
        base_navn_uten_endelse = os.path.splitext(base_navn)[0]
        filendelse = os.path.splitext(kildefilsti)[1].lower()
        
        # Bestem målfilsti
        if målmappesti is None:
            # Hvis ingen målmappe er angitt, bruk samme mappe med _geo.parquet
            dir_navn = os.path.dirname(kildefilsti)
            målfilsti = os.path.join(dir_navn, f"{base_navn_uten_endelse}_geo.parquet")
        else:
            # Hvis målmappe er angitt, plasser filen der
            målfilsti = os.path.join(målmappesti, f"{base_navn_uten_endelse}.parquet")
        
        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(målfilsti), exist_ok=True)
        
        gdf = None
        
        # Behandle filen basert på filtype
        if filendelse == '.parquet':
            # Sjekk først om filen er en vanlig parquet-fil
            df = pd.read_parquet(kildefilsti)
            
            # Sjekk etter lat/long kolonner
            if 'longitude' in df.columns and 'latitude' in df.columns:
                gdf = gpd.GeoDataFrame(
                    df, 
                    geometry=gpd.points_from_xy(df.longitude, df.latitude),
                    crs="EPSG:4326"
                )
            
            # Sjekk etter potensielle geometrikolonner
            if gdf is None:
                geom_kolonner = [col for col in df.columns if 'geom' in col.lower() 
                              or 'coord' in col.lower() 
                              or 'point' in col.lower() 
                              or 'polygon' in col.lower()
                              or 'linestring' in col.lower()
                              or 'wkt' in col.lower()]
                
                for col in geom_kolonner:
                    try:
                        # Prøv å konvertere fra WKT-format hvis det er en tekstkolonne
                        if df[col].dtype == 'object':
                            from shapely import wkt
                            geom = df[col].apply(wkt.loads)
                            gdf = gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
                            break
                    except Exception:
                        pass
        
        elif filendelse in ['.geojson', '.json']:
            # Les GeoJSON-filen uten å spesifisere driver for å unngå advarsel
            gdf = gpd.read_file(kildefilsti)
            
        elif filendelse == '.shp':
            # Les shapefile
            gdf = gpd.read_file(kildefilsti)
            
        elif filendelse == '.gpkg':
            # Les GeoPackage
            gdf = gpd.read_file(kildefilsti, driver='GPKG')
            
        elif filendelse == '.csv':
            # Prøv å lese CSV med lat/long kolonner
            df = pd.read_csv(kildefilsti)
            
            # Sjekk for vanlige lat/long kolonnenavn
            lat_kolonner = ['latitude', 'lat', 'y', 'breddegrad']
            lon_kolonner = ['longitude', 'long', 'lon', 'x', 'lengdegrad']
            
            lat_col = next((col for col in lat_kolonner if col in df.columns), None)
            lon_col = next((col for col in lon_kolonner if col in df.columns), None)
            
            if lat_col and lon_col:
                gdf = gpd.GeoDataFrame(
                    df, 
                    geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
                    crs="EPSG:4326"
                )
        
        # Hvis vi har en gyldig geodataframe, lagre som GeoParquet
        if gdf is not None:
            gdf.to_parquet(målfilsti)
            return målfilsti
        
        return False
        
    except Exception as e:
        print(f"Feil ved konvertering av {kildefilsti}: {e}")
        return False

def finn_nye_filer(rå_mappesti, prosessert_mappesti, konverteringslogg_sti=None):
    """Finner filer som ikke har blitt konvertert ennå"""
    # Støttede filformater
    støttede_formater = ['.parquet', '.geojson', '.json', '.shp', '.gpkg', '.csv']
    
    # Hent alle allerede konverterte filer
    konverterte_filer = set()
    if os.path.exists(prosessert_mappesti):
        for root, _, filer in os.walk(prosessert_mappesti):
            for fil in filer:
                if fil.endswith('.parquet'):
                    base_navn = os.path.splitext(fil)[0]
                    konverterte_filer.add(base_navn)
    
    # Last inn konverteringsloggen hvis den finnes
    konverterte_logger = {}
    if konverteringslogg_sti and os.path.exists(konverteringslogg_sti):
        try:
            with open(konverteringslogg_sti, 'r') as f:
                konverterte_logger = json.load(f)
        except Exception:
            pass
    
    # Finn alle filer som må konverteres
    nye_filer = []
    for root, _, filer in os.walk(rå_mappesti):
        for fil in filer:
            filsti = os.path.join(root, fil)
            filendelse = os.path.splitext(fil)[1].lower()
            base_navn = os.path.splitext(fil)[0]
            
            # Sjekk om filen har et støttet format
            if filendelse in støttede_formater:
                # Parquet-filer trenger en ekstra sjekk
                if filendelse == '.parquet':
                    # Ikke ta med filer som allerede er GeoParquet
                    if er_geoparquet(filsti):
                        continue
                    # Ikke ta med filer som allerede har _geo suffix
                    if base_navn.endswith('_geo'):
                        continue
                
                # Sjekk om filen allerede er konvertert
                if base_navn not in konverterte_filer:
                    # Sjekk om filen har endret seg siden sist konvertering
                    if base_navn in konverterte_logger:
                        sist_endret_tid = os.path.getmtime(filsti)
                        sist_konvertert_tid = konverterte_logger[base_navn].get('tidspunkt', 0)
                        if sist_endret_tid <= sist_konvertert_tid:
                            continue
                    
                    nye_filer.append(filsti)
    
    return nye_filer

def opprett_mappestruktur(rotmappe):
    """Oppretter den anbefalte mappestrukturen"""
    mapper = [
        os.path.join(rotmappe, 'raw'),
        os.path.join(rotmappe, 'processed'),
        os.path.join(rotmappe, 'archive')
    ]
    
    for mappe in mapper:
        os.makedirs(mappe, exist_ok=True)
    
    return {
        'raw': mapper[0],
        'processed': mapper[1],
        'archive': mapper[2]
    }

def oppdater_konverteringslogg(konverteringslogg_sti, fil, målfilsti):
    """Oppdaterer konverteringsloggen med ny filinfo"""
    konverterte_logger = {}
    
    # Last inn eksisterende logg hvis den finnes
    if os.path.exists(konverteringslogg_sti):
        try:
            with open(konverteringslogg_sti, 'r') as f:
                konverterte_logger = json.load(f)
        except Exception:
            pass
    
    # Legg til ny informasjon
    base_navn = os.path.splitext(os.path.basename(fil))[0]
    konverterte_logger[base_navn] = {
        'kildesti': fil,
        'målsti': målfilsti,
        'tidspunkt': datetime.datetime.now().timestamp(),
        'dato': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Lagre oppdatert logg
    with open(konverteringslogg_sti, 'w') as f:
        json.dump(konverterte_logger, f, indent=2)

def behandle_alle_filer_i_mappe(rotmappesti, opprett_struktur=True):
    """Behandler alle geografiske filer i angitt mappe og konverterer til GeoParquet."""
    # Sjekk om katalogen eksisterer
    if not os.path.exists(rotmappesti):
        print(f"FEIL: Katalogen '{rotmappesti}' eksisterer ikke.")
        return None
    
    mappestruktur = None
    rå_mappesti = rotmappesti
    prosessert_mappesti = rotmappesti
    
    # Opprett mappestruktur hvis ønsket
    if opprett_struktur:
        mappestruktur = opprett_mappestruktur(rotmappesti)
        rå_mappesti = mappestruktur['raw']
        prosessert_mappesti = mappestruktur['processed']
        print(f"Opprettet mappestruktur:\n- Raw: {rå_mappesti}\n- Processed: {prosessert_mappesti}")
    
    # Konverteringslogg-sti
    konverteringslogg_sti = os.path.join(rotmappesti, 'conversion_log.json')
    
    # Finn nye filer som trenger konvertering
    nye_filer = finn_nye_filer(rå_mappesti, prosessert_mappesti, konverteringslogg_sti)
    
    resultater = {
        "allerede_geoparquet": [],
        "konvertert": [],
        "feilet": [],
        "alle_filer": []
    }
    
    # Konverter filer
    for filsti in nye_filer:
        resultater["alle_filer"].append(filsti)
        
        # Konverter filen
        målfilsti = konverter_til_geoparquet(filsti, prosessert_mappesti)
        
        if målfilsti:
            resultater["konvertert"].append(filsti)
            oppdater_konverteringslogg(konverteringslogg_sti, filsti, målfilsti)
        else:
            resultater["feilet"].append(filsti)
    
    # Finn allerede eksisterende GeoParquet-filer
    for root, _, filer in os.walk(rå_mappesti):
        for fil in filer:
            if fil.endswith('.parquet'):
                filsti = os.path.join(root, fil)
                if er_geoparquet(filsti) and filsti not in resultater["alle_filer"]:
                    resultater["allerede_geoparquet"].append(filsti)
                    resultater["alle_filer"].append(filsti)
    
    # Oppsummering
    print("\n" + "="*50)
    print("OPPSUMMERING")
    print("="*50)
    print(f"Katalog: {rotmappesti}")
    if mappestruktur:
        print(f"Mappestruktur: Raw={rå_mappesti}, Processed={prosessert_mappesti}")
    print(f"Totalt antall geografiske filer funnet: {len(resultater['alle_filer'])}")
    print(f"Antall parquet-filer som allerede var GeoParquet: {len(resultater['allerede_geoparquet'])}")
    print(f"Antall filer konvertert til GeoParquet: {len(resultater['konvertert'])}")
    print(f"Antall filer som ikke kunne konverteres: {len(resultater['feilet'])}")
    
    if resultater["konvertert"]:
        print("\nKonverterte filer:")
        for fil in resultater["konvertert"]:
            print(f"- {fil}")
    
    if resultater["feilet"]:
        print("\nFiler som ikke kunne konverteres:")
        for fil in resultater["feilet"]:
            print(f"- {fil}")
    
    return resultater

# Angi rotmappen som skal behandles
rotmappe = "/Users/christine/00 Bachelortesting/usecase1_parquet/Data"
resultater = behandle_alle_filer_i_mappe(rotmappe, opprett_struktur=True)


Opprettet mappestruktur:
- Raw: /Users/christine/00 Bachelortesting/usecase1_parquet/Data/raw
- Processed: /Users/christine/00 Bachelortesting/usecase1_parquet/Data/processed

OPPSUMMERING
Katalog: /Users/christine/00 Bachelortesting/usecase1_parquet/Data
Mappestruktur: Raw=/Users/christine/00 Bachelortesting/usecase1_parquet/Data/raw, Processed=/Users/christine/00 Bachelortesting/usecase1_parquet/Data/processed
Totalt antall geografiske filer funnet: 55
Antall parquet-filer som allerede var GeoParquet: 55
Antall filer konvertert til GeoParquet: 0
Antall filer som ikke kunne konverteres: 0
